In [2]:
import pandas as pd
import numpy as np
import requests
import re
import json
from bs4 import BeautifulSoup
import time
from http.client import RemoteDisconnected
from urllib3.exceptions import ProtocolError, NewConnectionError, MaxRetryError
from urllib.error import HTTPError
from socket import gaierror

In [3]:
locations = {
    "dong-nai": "Đồng Nai",
    "binh-duong": "Bình Dương",
    "tp-ho-chi-minh": "TP Hồ Chí Minh"
}

type_real_estates = {
    "mua-ban-can-ho-chung-cu": "Căn hộ/Chung cư",
    "mua-ban-nha-dat": "Nhà ở",
    "mua-ban-dat": "Đất",
}

attribute_map = [
    "ward",
    "area",
    "region",
    "address",
    "property_status",
    "price_m2",
    "direction",
    "balconydirection",
    "property_legal_document",
    "size",
    "floornumber",
    "apartment_type",
    "furnishing_sell",
    "apartment_feature",
    "rooms",
    "toilets",
    "floors",
    "house_type",
    "living_size",
    "width",
    "length",
    "land_type",
    "property_road_condition",
    "land_feature",
    "property_back_condition",
]

base_url = "https://nha.chotot.com/{}/{}?page={}"
api = "https://gateway.chotot.com/v1/public/ad-listing/{}"


In [12]:
def parse_data_api(type_name, id, data, json_text):
    json_data = json.loads(json_text)

    if "ad" not in json_data or "parameters" not in json_data:
        return

    if "type_name" in json_data["ad"] and json_data["ad"]["type_name"] == "Cần mua":
        return

    data["type_real_estate"].append(type_name)
    data["id"].append(id)

    # Thuộc tính giá (price) và thời gian đăng tin (list_time) nằm trong ad
    data["price"].append(json_data["ad"]["price"] if "price" in json_data["ad"] else np.nan)
    data["list_time"].append(json_data["ad"]["list_time"] if "list_time" in json_data["ad"] else np.nan)

    # Các thuộc tính trong parameter sẽ được thêm vào đây
    attr = ["id", "type_real_estate", "price", "list_time"]

    # Map các thuộc tính trong parameter vào dataset
    for para in json_data["parameters"]:
        if "id" in para:
            para_id = para["id"]
            if "value" in para and para_id in attribute_map:
                value = para["value"]
                data[para_id].append(value)
                attr.append(para_id)

    # Các thuộc tính missing đặt giá trị là nan
    for attribute in attribute_map:
        if attribute not in attr:
            data[attribute].append(np.nan)


In [13]:
data = {
    "id": [],
    "list_time": [],
    "type_real_estate": [],
    "ward": [],
    "area": [],
    "region": [],
    "address": [],
    "property_status": [],
    "price_m2": [],
    "direction": [],
    "balconydirection": [],
    "property_legal_document": [],
    "size": [],
    "floornumber": [],
    "apartment_type": [],
    "furnishing_sell": [],
    "apartment_feature": [],
    "rooms": [],
    "toilets": [],
    "floors": [],
    "house_type": [],
    "living_size": [],
    "width": [],
    "length": [],
    "land_type": [],
    "property_road_condition": [],
    "land_feature": [],
    "property_back_condition": [],
    "price": []
}


In [15]:
total = 0
for location, location_name in locations.items():
    data = {
        "id": [],
        "list_time": [],
        "type_real_estate": [],
        "ward": [],
        "area": [],
        "region": [],
        "address": [],
        "property_status": [],
        "price_m2": [],
        "direction": [],
        "balconydirection": [],
        "property_legal_document": [],
        "size": [],
        "floornumber": [],
        "apartment_type": [],
        "furnishing_sell": [],
        "apartment_feature": [],
        "rooms": [],
        "toilets": [],
        "floors": [],
        "house_type": [],
        "living_size": [],
        "width": [],
        "length": [],
        "land_type": [],
        "property_road_condition": [],
        "land_feature": [],
        "property_back_condition": [],
        "price": []
    }

    print(f'{location_name}:')
    for type_real_estate, type_name in type_real_estates.items():
        count = 0
        page = 1
        while True:
            url = base_url.format(location, type_real_estate, page)

            result = requests.get(url)

            if result.status_code == 200:
                soup = BeautifulSoup(result.text, "html.parser")
                items = soup.find_all("a", class_="AdItem_adItem__2O28x", href=True)

            # Thoát khỏi vòng lặp khi đã crawl dữ liệu hết loại này
            if len(items) == 0:
                break

            for item in items:
                href = item["href"]
                id_extract = re.findall(r"(\d+).htm", href)

                if len(id_extract) == 0:
                    break

                while True:
                    try:
                        result_item = requests.get(api.format(id_extract[0]))
                        if result_item.status_code == 200:
                            json_text = result_item.text
                            parse_data_api(type_name, id_extract[0], data, json_text)
                            count += 1
                            print("\r\tCrawled page {}: {} items".format(page, count), end="", flush=True)
                            break

                    except (ConnectionError, ProtocolError, RemoteDisconnected, HTTPError,
                            gaierror, NewConnectionError, MaxRetryError) as e:
                        time.sleep(90)
                        print(e)
                        continue

            page += 1

        total += count
        print(f'\r\t-{type_name} ({page-1} pages): {count} items', flush=True)

        df = pd.DataFrame(data)
        df.to_csv(f'{location} {type_real_estate}.csv', index=False)

print(f'Total = {total} items')

Đồng Nai:
	-Căn hộ/Chung cư (11 pages): 218 items
	-Nhà ở (116 pages): 2312 items
	-Đất (318 pages): 6354 items
Bình Dương:
	-Căn hộ/Chung cư (120 pages): 2392 items
	-Nhà ở (203 pages): 4069 items
	-Đất (310 pages): 6192 items
TP Hồ Chí Minh:
	-Căn hộ/Chung cư (524 pages): 10481 items
	-Nhà ở (2436 pages): 48715 items
	-Đất (757 pages): 15139 items
Total = 95872 items


In [ ]:
df_merge = []
for location, location_name in locations.items():
    file = f'{location} mua-ban-dat.csv'
    df_merge.append(pd.read_csv(file))

/home/daivo/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7,10,14,15,16,17,18,19,20,21,24,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df_final = pd.concat(df_merge)
df_final.head()

,id,list_time,type_real_estate,ward,area,region,address,property_status,price_m2,direction,...,floors,house_type,living_size,width,length,land_type,property_road_condition,land_feature,property_back_condition,price
0,88869349,1.636102e+12,Căn hộ/Chung cư,Phường Tam Hòa,Thành phố Biên Hòa,Đồng Nai,"Đường Đồng Khởi, Phường Tam Hòa, Thành phố Biê...",Đã bàn giao,"16,43 triệu/m²",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.150000e+09
1,89325874,1.636101e+12,Căn hộ/Chung cư,Phường Thống Nhất,Thành phố Biên Hòa,Đồng Nai,"Đường Võ Thị Sáu, Phường Thống Nhất, Thành phố...",Đã bàn giao,"32,29 triệu/m²",Bắc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.550000e+09
2,87924820,1.636098e+12,Căn hộ/Chung cư,Phường Hố Nai,Thành phố Biên Hòa,Đồng Nai,"368 Đường Xa Lộ Hà Nội, Phường Hố Nai, Thành p...",Chưa bàn giao,"5,37 triệu/m²",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.600000e+08
3,89311358,1.636080e+12,Căn hộ/Chung cư,Phường Hố Nai,Thành phố Biên Hòa,Đồng Nai,"368 Đường Xa Lộ Hà Nội, Phường Hố Nai, Thành p...",Chưa bàn giao,"24,29 triệu/m²",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.700000e+09
4,89200226,1.636080e+12,Căn hộ/Chung cư,Phường Thống Nhất,Thành phố Biên Hòa,Đồng Nai,"Đường Võ Thị Sáu, Phường Thống Nhất, Thành phố...",Đã bàn giao,208.333 đ/m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+07


In [6]:
df_final.shape

(95413, 29)

In [ ]:
df_final.to_csv('chotot.csv', index=False)